# 15 차시 - 본문 실습

## 엑셀 3대 기능 - 필터링/소트, VLOOKUP, 피벗



# 필터링과 소트
* df.query()
* df.sort_values()


데이터 준비

* "관심종목.xlsx" http://bit.ly/30PQjbI
* "종목별 팩터 데이터.xlsx" http://bit.ly/34e2T6E
* "광고비 데이터 3사.xlsx" http://bit.ly/32gVlyc

In [ ]:
import pandas as pd

df_팩터 = pd.read_excel('종목별 팩터 데이터.xlsx', dtype={'종목코드':str})
df_팩터 = df_팩터.set_index('종목코드')
df_팩터

In [ ]:
df_팩터[df_팩터['PER'] <= 10]

In [ ]:
df = df_팩터[df_팩터['PER'] <= 10].sort_values('배당수익률', ascending=False)
df

In [ ]:
# 조건에 해당하는 상위 3개 종목

df[['종목명']].head(3)

## 필터링을 간편하게 - df.query()

In [ ]:
df_팩터.query("PER <= 10 and 배당수익률 > 2.0")

## 코드 기준으로 병합 하기

In [ ]:
df_관심종목 = pd.read_excel('관심종목.xlsx', dtype={'종목코드':str})
df_관심종목 = df_관심종목.set_index('종목코드') 
df_관심종목

In [ ]:
df_팩터 = pd.read_excel('종목별 팩터 데이터.xlsx', dtype={'종목코드':str})
df_팩터 = df_팩터.set_index('종목코드')
df_팩터

## 인덱스를 활용한 DataFrame 병합

In [ ]:
# 개별 컬럼 복사 (없으면 생성된다)
df_관심종목['PER'] = df_팩터['PER']
df_관심종목

In [ ]:
df_관심종목[['PER', 'PBR', '배당수익률']] = df_팩터[['PER', 'PBR', '배당수익률']]
df_관심종목

In [ ]:
df_관심종목 = df_관심종목.dropna()
df_관심종목

## merge() 함수로 병합

In [ ]:
df_관심종목 = pd.read_excel('관심종목.xlsx', dtype={'종목코드':str})
df_팩터 = pd.read_excel('종목별 팩터 데이터.xlsx', dtype={'종목코드':str})

In [ ]:
pd.merge(df_관심종목, df_팩터)

In [ ]:
pd.merge(df_관심종목, df_팩터, how='outer')

## 피벗 테이블

In [ ]:
import pandas as pd

df = pd.read_excel('광고비 데이터 3사.xlsx', dtype={'종목코드':str})
df.head(5)

In [ ]:
table = pd.pivot_table(df, values='total', index='date', columns='name')
table

In [ ]:
table = pd.pivot_table(df, values='total', index='date', columns='name', aggfunc='sum', margins=True)
table

## OpenPyXL

In [ ]:
데이터 준비

* "2017년 광고비 - 삼성전자.xlsx" https://goo.gl/3JdWsL
* "2017년 광고비 - LG전자.xlsx" https://goo.gl/CzqLjD


## 워크북 읽기와 시트


In [ ]:
import openpyxl

wb = openpyxl.load_workbook('2017년 광고비 - 삼성전자.xlsx')
wb

In [ ]:
# 모든 시트 이름들 얻기
wb.sheetnames

In [ ]:
# 시트 이름으로 시트 얻기
sheet = wb['Sheet1']
sheet

In [ ]:
# 활성화 시트 얻기
sheet = wb.active
sheet

### 셀에 접근

In [ ]:
sheet['A2'].value 
# '2017-01'

In [ ]:
sheet['B1'].value 
# 'name'

In [ ]:
# 값 할당하기
sheet['C1'].value = 43700

In [ ]:
sheet.cell(row = 1, column = 3).value 
# 43700

# 두 엑셀 파일 읽어 하나로 합치기

In [ ]:
import pandas as pd

df_삼성전자 = pd.read_excel('2017년 광고비 - 삼성전자.xlsx')
df_삼성전자.set_index('date', inplace=True)

df_LG전자 = pd.read_excel('2017년 광고비 - LG전자.xlsx')
df_LG전자.set_index('date', inplace=True)

In [ ]:
df_merge = pd.DataFrame()

df_merge['삼성전자'] = df_삼성전자['total']
df_merge['LG전자'] = df_LG전자['total']

In [ ]:
df_merge.to_excel('merged_01.xlsx')

# 계산 및 계산 결과 저장

In [ ]:
import openpyxl

# 엑셀 파일 열고, 시트 얻기
wb = openpyxl.load_workbook('merged_01.xlsx')
sheet = wb.active
sheet

In [ ]:
# 삼성전자 컬럼
삼성전자_월광고비 = [row[0].value for row in sheet['B2':'B13']]
삼성전자_월광고비

In [ ]:
# 합계
sum(삼성전자_월광고비)

In [ ]:
sheet['B14'].value = sum(삼성전자_월광고비)
sheet['B14'].value

In [ ]:
# '합계'
sheet['A14'].value = '합계'

In [ ]:
# 합계 계산
sum_삼성전자 = sum([row[0].value for row in sheet['B2':'B13']])
sheet['B14'].value = sum_삼성전자

sum_LG전자 = sum([row[0].value for row in sheet['C2':'C13']])
sheet['C14'].value = sum_LG전자

In [ ]:
wb.save("merged_02.xlsx")

# 수식 넣기

In [ ]:
# 합계 계산
sheet['B14'].value = '=SUM(B2:B13)'
sheet['C14'].value = '=SUM(C2:C13)'

In [ ]:
wb.save("merged_03.xlsx")

# 셀에 스타일 지정

In [ ]:
from openpyxl.styles import Font, Alignment
from openpyxl.styles import Border, Side, Color, PatternFill

# Font: '맑은 고딕', 크기 15, 굵게
font_15 = Font(name='맑은 고딕', size=15, bold=True)

# Alignment: 가로 세로 , 가운데 정렬
align_center = Alignment(horizontal='center', vertical='center')
align_vcenter = Alignment(vertical='center')

# Border: 테두리 지정
border_thin = Border(
    left=Side(style='thin'), right=Side(style='thin'), 
    top=Side(style='thin'), bottom=Side(style='thin'))

# PatternFill: 셀 색상 지정
fill_orange = PatternFill(patternType='solid', fgColor=Color('FFC000'))
fill_lightgrey = PatternFill(patternType='solid', fgColor=Color('D3D3D3'))

In [ ]:
cell_sum = sheet['A14'] # 합계 제목 셀

cell_sum.font = font_15
cell_sum.alignment = align_center
cell_sum.border = border_thin
cell_sum.fill = fill_orange

In [ ]:
wb.save("merged_04.xlsx")

# 범위에 스타일 지정

In [ ]:
# 범위(range)
sheet['B2:C14']

In [ ]:
for row in sheet['B2:C14']:
    for cell in row:
        cell.border = border_thin
        cell.number_format = '0.00'
        
for row in sheet['B14:C14']:
    for cell in row:
        cell.alignment = align_vcenter
        cell.fill = fill_orange

In [ ]:
wb.save("merged_05.xlsx")

# 차트 추가 하기


In [ ]:
from openpyxl.chart import Reference, Series, BarChart

chart = BarChart()
chart.title = '2017년 월별 광고비 (억원)'

vals = Reference(sheet, range_string= 'Sheet1!B1:B13')
series = Series(vals, title="삼성전자")
chart.append(series)

vals = Reference(sheet, range_string= 'Sheet1!C1:C13')
series = Series(vals, title="LG전자")
chart.append(series)

sheet.add_chart(chart, 'E1')

In [ ]:
wb.save("merged_06.xlsx")

## xlwings 소개

xlwings 시작과 종료

```python
import xlwings as xw

wb = xw.Book()  # 새로운 워크북
wb = xw.Book('FileName.xlsx')  # 현재 작업 디렉토리의 'FileName.xlsx' 파일
wb = xw.Book(r'C:\path\to\file.xlsx')  # 특정 디렉토리(C:\path\to)의 'file.xlsx' 파일
```

In [ ]:
import xlwings as xw

In [ ]:
wb = xw.Book() # 워크북을 새로 생성합니다 

In [ ]:
wb.close() # 워크북을 닫습니다

## 셀에 데이터 쓰기/읽기

In [ ]:
wb = xw.Book() # 워크북을 새로 생성합니다 

In [ ]:
wb.sheets

In [ ]:
sht = wb.sheets['Sheet1']
sht

In [ ]:
sht.range('A1')

시트에서 영역(Range)를 선택합니다

In [ ]:
# 셀에 값 쓰기/읽기
sht.range('A1').value = 10
sht.range('A2').value = 20

sht.range('A1').value

## Range를 지정하는 방법들
```python
Range('A1')          
Range('A1:C3')       
Range((1,2))
Range((1,1), (3,3))
Range('NamedRange')
```

In [ ]:
sht = xw.sheets.active
sht.range('A1:C3').value

In [ ]:
xw.Range('A1:C3').value

In [ ]:
# current_region (윤곽) Ctrl-* 에 해당하는 영역
xw.Range('A1').current_region.value

## 셀에 값 쓰기/읽기와 데이터 타입

In [ ]:
# 셀에 값 쓰기/읽기
sht.range('A1').value = 10
sht.range('A2').value = 20.0
sht.range('A3').value = 'Hello, 파이썬'
sht.range('A4').value = '2020-01-01'

In [ ]:
for c in ['A1', 'A2', 'A3', 'A4',]:
    print(sht.range(c).value, type(sht.range(c).value))

In [ ]:
sht.range('A1:A2').options(ndim=1).value

In [ ]:
sht.range('A1:A2').options(ndim=2).value

In [ ]:
sht.range('A1:A2').value

In [ ]:
sht.range('A1:A2').options(numbers=int).value

In [ ]:
from datetime import datetime, date

sht.range('A4').options(dates=date).value

In [ ]:
# options(empty=)
sht.range('A5').options(empty='NA').value

In [ ]:
# options(transpose=True/False)
sht.range('B1').options(transpose=True).value = [1, 2, 3]

In [ ]:
sht.range('B1').options(transpose=False).value = [1, 2, 3]

In [ ]:
# options(expand='table') vertical, horizontal
sht.range('A1').options(expand='table').value

In [ ]:
sht.range('A1').options(expand='down').value

In [ ]:
sht.range('A1').options(expand='horizontal').value

## 영역 내용 지우기

In [ ]:
sht.range('A1:E5').clear_contents()

## 셀에 2차원 데이터 쓰기/읽기

In [ ]:
sht.range('A1').value = [['A','B','C'], [10, 20, 30], [40, 50, 60]]

In [ ]:
sht.range('A1').value

In [ ]:
sht.range('A1').expand().value

In [ ]:
sht.range('A1').expand().value #기본값='table'

In [ ]:
sht.range('A1').expand('table').value 

In [ ]:
sht.range('A1').expand('down').value

In [ ]:
sht.range('A1').expand('right').value

In [ ]:
# options를 사용하는 경우
print(sht.range('A1').options(expand='table').value)
print(sht.range('A1').options(expand='down').value)
print(sht.range('A1').options(expand='right').value)

# 시트 추가/삭제


In [ ]:
# 자동으로 "Sheet+번호" 형식의 이름이 부여된다
wb.sheets.add()

In [ ]:
# 이름을 지정하여 시트를 추가
new_sheet = wb.sheets.add(name='sheet_for_test')
new_sheet

In [ ]:
# 시트 목록
wb.sheets

In [ ]:
# 'sheet_for_test' 시트 삭제
new_sheet.delete()

In [ ]:
# 'Sheet2' 이름을 지정하여 시트 삭제
wb.sheets['Sheet2'].delete()

# 엑셀 파일 저장

In [ ]:
# 저장
wb.save('sample_01.xlsx')

In [ ]:
# 닫기
wb.close()

---
## 워크북 열기

In [ ]:
import xlwings as xw

wb = xw.Book()
sht = wb.sheets.active

## DataFrame 읽기/쓰기

options에서 인덱스와 헤더 사용여부를 지정할 수 있습니다

* index: 인덱스가 있는지 여부 (기본값:1)
* header: 헤더가 있는지 여부(기본값:1)


In [ ]:
import pandas as pd

In [ ]:
# DataFrame to 엑셀

df = pd.DataFrame([[10, 20, 30], [40, 50, 60]], columns=['A','B','C'])
df

인덱스를 지정하지 않으면 기본적으로 RangeIndex가 지정됩니다.

In [ ]:
sht.range('A1').value = df

In [ ]:
# 현재 시트의 내용을 모두 삭제
sht.range('A1').current_region.clear_contents()

In [ ]:
sht.range('A1').options(index=False).value = df

In [ ]:
# 엑셀 to DataFrame
df = sht.range('A1').options(pd.DataFrame, expand='table').value
df

In [ ]:
# 엑셀 to DataFrame
df = sht.range('A1').options(pd.DataFrame, index=False, expand='table', copy=True).value
df

In [ ]:
# 엑셀 닫기
app = xw.apps.active
app.quit()

## 데이터 준비 (인터넷망 경우)

In [ ]:
## Finance-DataReader 설치
!pip install -q Finance-DataReader

In [ ]:
import FinanceDataReader as fdr

df = fdr.DataReader('KS11', '2020-01-01','2020-01-31')
df.to_csv('KOSPI-202001.csv')

## 데이터 준비 (업무망 경우: 인터넷 불가환경)

In [ ]:
%%writefile KOSPI-202001.csv
Date,Close,Open,High,Low,Volume,Change
2020-01-02,2175.17,2201.21,2202.32,2171.84,494680000.0,-0.0102
2020-01-03,2176.46,2192.58,2203.38,2165.39,631560000.0,0.0006
2020-01-06,2155.07,2154.97,2164.42,2149.95,592670000.0,-0.0098
2020-01-07,2175.54,2166.6,2181.62,2164.27,568240000.0,0.0095
2020-01-08,2151.31,2156.27,2162.32,2137.72,913830000.0,-0.0111
2020-01-09,2186.45,2182.2,2186.45,2172.16,592600000.0,0.0163
2020-01-10,2206.39,2189.48,2206.92,2188.1,594540000.0,0.0091
2020-01-13,2229.26,2204.49,2230.43,2204.49,488290000.0,0.0104
2020-01-14,2238.88,2243.06,2250.79,2232.94,835900000.0,0.0043
2020-01-15,2230.98,2229.8,2238.21,2223.39,746160000.0,-0.0034999999999999996
2020-01-16,2248.05,2231.93,2249.09,2225.41,808330000.0,0.0077
2020-01-17,2250.57,2261.57,2268.71,2245.58,580820000.0,0.0011
2020-01-20,2262.64,2261.4,2277.23,2258.09,545490000.0,0.0054
2020-01-21,2239.69,2259.81,2266.31,2238.25,770320000.0,-0.0101
2020-01-22,2267.25,2238.53,2269.27,2235.13,608510000.0,0.0123
2020-01-23,2246.13,2254.54,2260.63,2241.94,509610000.0,-0.009300000000000001
2020-01-28,2176.72,2192.22,2199.93,2166.23,808880000.0,-0.030899999999999997
2020-01-29,2185.28,2188.18,2195.45,2172.33,692300000.0,0.0039000000000000003
2020-01-30,2148.0,2181.54,2187.45,2139.72,602840000.0,-0.0171
2020-01-31,2119.01,2160.6,2167.23,2119.01,710510000.0,-0.013500000000000002

# xlwings 엑셀 서식

## 데이터 읽기

In [ ]:
## CSV 파일을 읽기 (CSV)
import xlwings as xw
from xlwings import constants

wb = xw.Book('KOSPI-202001.csv')

## 서식 지정하기

In [ ]:
## 숫자 포맷 지정
sht.range('B:E').number_format = '0,0.0'
sht.range('F:F').number_format = '0,0'
sht.range('G:G').number_format = '0.00%'

In [ ]:
# 테두리
for cell in sht.range('A1').current_region:
    for border_id in range(7,12+1):
        cell.api.Borders(border_id).LineStyle = 1

In [ ]:
# 헤더 색상, 폰트스타일 지정
header_rgn = sht.range('A1').expand('right')

header_rgn.api.Font.Bold = True
header_rgn.api.Interior.ColorIndex = 6
header_rgn.api.HorizontalAlignment = constants.HAlign.xlHAlignCenter

In [ ]:
# 자동 정렬
sht.autofit()

## xlsx로 저장하기

새로운 워크북을 생성하여 새로운 이름을 부여하여 저장합니다

In [ ]:
sht.api.Copy() # 시트를 복사하여 새로운 워크북 생성

In [ ]:
xw.books[1].save('KOSPI_202001.xlsx')

In [ ]:
# 엑셀 닫기
app = xw.apps.active    
app.quit()

# xlwings - 엑셀 서식 다루기

## 워크북 열기

In [ ]:
import xlwings as xw

wb = xw.Book()
sht = wb.sheets.active

sht.range('A1').value = [['A','B','C'], [10, 0.2, 30], [40, 0.5, 60], [70, 0.8, 90]]

## 셀 포맷 지정하기

영역(range)의 number_format 속성에 숫자 형식 지정합니다

In [ ]:
sht.range('A:A').number_format = '0.0'
sht.range('B:B').number_format = '0,0.00%'
sht.range('C:C').number_format = '0,0원'

## API 프로퍼티 (확장 접근)
세밀한 접근을 위해 api 프로퍼티를 사용하여 다양한 조작을 수행합니다.

In [ ]:
sht.api

## 테두리 라인 그리기

In [ ]:
xw.Range('A1').current_region

In [ ]:
for cell in sht.range('A1').current_region:
    for border_id in range(7,12+1):
        cell.api.Borders(border_id).LineStyle = 1
        cell.api.Borders(border_id).Weight = 2

In [ ]:
# 7~11번은 각각 
sht.range('A1:C1').api.Borders(9).LineStyle = -4119

## Font 객체
폰트 크기(Size), 스타일(Bold, Italic), 색상(ColorIndex) 등을 지정할 수 있습니다.

In [ ]:
sht.range('1:4').api.Font.Size = 18

In [ ]:
header_rgn = sht.range('A1:C1')

header_rgn.api.Font.Size = 20
header_rgn.api.Font.FontStyle = 'Bold'
header_rgn.api.Font.ColorIndex = 3
header_rgn.api.Interior.ColorIndex = 6

In [ ]:
# 배경색 지정
# xw.utils.rgb_to_int((r<img src="https://i.imgur.com/5cxLoI7.png?1">,g,b)) - (r,g,b) 튜플을 int값으로 반환합니다
header_rgn.color = xw.utils.rgb_to_int((200, 200, 200))

## 텍스트 정렬
가로 세로 텍스트 맞춤

In [ ]:
from xlwings import constants

sht.range('A1:C1').api.HorizontalAlignment = constants.HAlign.xlHAlignCenter
sht.range('A1:C1').api.VerticalAlignment = constants.VAlign.xlVAlignCenter

In [ ]:
rgn = sht.range('A1:C3')
rgn

## 자동맞춤
```python
autofit(axis=None)
```
axis: 'c'=컬럼, 'r'=로우, 지정하지 않으면 컬럼과 로우 모두 자동맞춤


In [ ]:
# 자동 맞춤 (시트 전체)
wb.sheets['Sheet1'].autofit('c')
wb.sheets['Sheet1'].autofit('r')

In [ ]:
# 자동 맞춤 (영역)
rgn = sht.range('A1:C4')

rgn.columns.autofit()  # autofit column and row
rgn.rows.autofit()  # autofit column and row

## 자동줄바꿈 지정

In [ ]:
# 자동줄바꿈 지정
sht.range('A6').value = 'this is long long text'
sht.range('A6').api.WrapText = True

In [ ]:
sht.range('A6').api.WrapText = False

In [ ]:
# 닫기
wb.close()

# xlwings - 엑셀 차트 그리기
 

## KOSPI 데이터 읽기

In [ ]:
import xlwings as xw
wb = xw.Book('KOSPI_202001.xlsx')

In [ ]:
import pandas as pd

sht = wb.sheets[0]
df = sht.range('A1').options(pd.DataFrame, index=False, expand='table').value
df

## xlwings 차트그리는 순서
1. 차트 생성
2. 차트 데이터 선택
3. 차트 설정 (제목, 크기, 종류) 추가
 

In [ ]:
# 1. 차트 생성
chart = sht.charts.add()

In [ ]:
# 2. 차트 데이터 선택
chart.set_source_data(sht.range('B:B').expand('down'))

In [ ]:
# 3. 차트의 설정 (종류, 위치) 
chart.left = 400
chart.top = 50
chart.width = 500
chart.height = 300

In [ ]:
chart.chart_type = 'line'

## 차트 상세 설정
chart의 다양한 속성들을 다음을 참고 합니다.<br>
https://docs.microsoft.com/en-us/office/vba/api/excel.chart.application

In [ ]:
chart.api

In [ ]:
chart.api[1].HasLegend = False # 범례 켜기/끄기

In [ ]:
chart.api[1].ChartTitle.Text = 'KOSPI 2020-01' # 차트 title

In [ ]:
# 차트 위치를 셀위치로
chart.top = sht.range('I2').top
chart.left = sht.range('I2').left

In [ ]:
# 새로운 이름으로 저장하기

sht.api.Copy() # 시트를 복사하여 새로운 워크북 생성
xw.books[1].save('KOSPI_202001_with_chart.xlsx') # 워크북을 저장